In [34]:
import os
import yaml
import time
import optuna

from datasets import load_dataset, load_from_disk
from plaid.bridges.huggingface_bridge import huggingface_dataset_to_plaid, huggingface_description_to_problem_definition
from sklearn.pipeline import Pipeline
from sklearn.base import clone
from sklearn.model_selection import KFold
from sklearn.decomposition import PCA

from sklearn.utils.estimator_checks import  check_estimator

from sklearn.model_selection import GridSearchCV

from plaid.pipelines.sklearn_block_wrappers import WrappedPlaidSklearnTransformer, WrappedPlaidSklearnRegressor
from plaid.pipelines.plaid_blocks import PlaidTransformedTargetRegressor, PlaidColumnTransformer

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import joblib

import numpy as np

import warnings
warnings.filterwarnings('ignore', module='sklearn')

nb_cpus = os.cpu_count()
print("Number of CPUs:", nb_cpus)

Number of CPUs: 24


In [35]:

with open("config_5.yml") as f:
    config = yaml.safe_load(f)

global_params = config["global"]


start = time.time()
hf_dataset = load_dataset(global_params['dataset_path'], split="all_samples")
# hf_dataset = load_from_disk(global_params['dataset_path'])
# print(f"Loading dataset from HuggingFace Hub took: {time.time() - start:.2g} seconds")

prob_def = huggingface_description_to_problem_definition(hf_dataset.description)

train_split = prob_def.get_split(global_params['train_split_name'])[:24]
dataset_train, _ = huggingface_dataset_to_plaid(hf_dataset, ids = train_split, processes_number = int(nb_cpus/4))#os.cpu_count())

test_split = prob_def.get_split(global_params['test_split_name'])[:24]
dataset_test, _ = huggingface_dataset_to_plaid(hf_dataset, ids = test_split, processes_number = int(nb_cpus/4))#os.cpu_count())

del hf_dataset

Converting huggingface dataset to plaid dataset...


100%|██████████| 24/24 [00:01<00:00, 19.57it/s]

Converting huggingface dataset to plaid dataset...



100%|██████████| 24/24 [00:00<00:00, 36.56it/s]


In [36]:
# from_features_identifier
all_feature_id = config['input_scalar_scaler']['plaid_params']['in_features_identifiers']+config['pca_nodes']['plaid_params']['in_features_identifiers']+config['pca_mach']['plaid_params']['in_features_identifiers']
print("dataset_train[0] =", dataset_train[0])
print("dataset_test[0] =", dataset_test[0])
dataset_train = dataset_train.from_features_identifier(all_feature_id)
dataset_test = dataset_test.from_features_identifier(all_feature_id)
print("dataset_train[0] =", dataset_train[0])
print("dataset_test[0] =", dataset_test[0])

dataset_train[0] = Sample(8 scalars, 0 time series, 1 timestamp, 8 fields)
dataset_test[0] = Sample(2 scalars, 0 time series, 1 timestamp, 1 field)
dataset_train[0] = Sample(2 scalars, 0 time series, 1 timestamp, 1 field)
dataset_test[0] = Sample(2 scalars, 0 time series, 1 timestamp, 1 field)


In [37]:
# check_estimator(WrappedPlaidSklearnTransformer())

In [38]:
wrapped_transf = WrappedPlaidSklearnTransformer(MinMaxScaler(), **config['input_scalar_scaler']['plaid_params'])
clone(wrapped_transf)

,sklearn_block,MinMaxScaler()
,in_features_identifiers,"[{'name': 'angle_in', 'type': 'scalar'}, {'name': 'mach_out', 'type': 'scalar'}]"
,out_features_identifiers,None
,feature_range,"(0, ...)"
,copy,True
,clip,False


In [ ]:
preprocessor = PlaidColumnTransformer([
    ('input_scalar_scaler', WrappedPlaidSklearnTransformer(MinMaxScaler(), **config['input_scalar_scaler']['plaid_params'])),
    ('pca_nodes', WrappedPlaidSklearnTransformer(PCA(**config['pca_nodes']['sklearn_block_params']), **config['pca_nodes']['plaid_params'])),
], remainder_feature_ids = config['pca_mach']['plaid_params']['in_features_identifiers'])
preprocessor

,plaid_transformers,"[('input_scalar_scaler', ...), ('pca_nodes', ...)]"
,remainder_feature_id,"[{'base_name': 'Base_2_2', 'name': 'mach', 'type': 'field'}]"
,feature_range,"(0, ...)"
,copy,True
,clip,False
,n_components,2
,copy,True
,whiten,False
,svd_solver,'auto'
,tol,0.0
,iterated_power,'auto'


In [40]:
params = {'pca_nodes__sklearn_block__n_components': 5}
index = 5
preprocessor_ = clone(preprocessor)
preprocessor_.set_params(**params)
dataset_pred = preprocessor_.fit_transform(dataset_train)
print("angle_in =", dataset_pred[index].get_scalar("angle_in"))
print("mach_out =", dataset_pred[index].get_scalar("mach_out"))
print("reduced_nodes_* =", dataset_pred[index].get_scalar("reduced_nodes_*"))

angle_in = 0.6386494252873546
mach_out = 0.15684931506849198
reduced_nodes_* = [-31.13256395  26.35027297  -0.97654601  -0.93225342   0.12015363]


In [41]:
print(preprocessor_.get_params()['pca_nodes__sklearn_block__n_components'])
print(preprocessor_.transformers[1][1].get_params()['sklearn_block__n_components'])
print(preprocessor_.transformers[1][1].sklearn_block.get_params())

5
5
{'copy': True, 'iterated_power': 'auto', 'n_components': 5, 'n_oversamples': 10, 'power_iteration_normalizer': 'auto', 'random_state': None, 'svd_solver': 'auto', 'tol': 0.0, 'whiten': False}


In [42]:
postprocessor = WrappedPlaidSklearnTransformer(PCA(**config['pca_mach']['sklearn_block_params']), **config['pca_mach']['plaid_params'])
postprocessor

,sklearn_block,PCA(n_components=2)
,in_features_identifiers,"[{'base_name': 'Base_2_2', 'name': 'mach', 'type': 'field'}]"
,out_features_identifiers,"[{'name': 'reduced_mach_*', 'type': 'scalar'}]"
,n_components,2
,copy,True
,whiten,False
,svd_solver,'auto'
,tol,0.0
,iterated_power,'auto'
,n_oversamples,10
,power_iteration_normalizer,'auto'


In [43]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ConstantKernel
from sklearn.multioutput import MultiOutputRegressor


kernel = ConstantKernel() * Matern(length_scale_bounds=(1e-8, 1e8), nu = 2.5) + WhiteKernel(noise_level_bounds=(1e-8, 1e8))
# kernel = Matern(nu = 2.5)

gpr = GaussianProcessRegressor(
    kernel=kernel,
    optimizer='fmin_l_bfgs_b',
    n_restarts_optimizer=1,
    random_state=42)

reg = MultiOutputRegressor(gpr)


def length_scale_init(X):
    return np.ones(X.shape[1])

dynamics_params_factory = {'estimator__kernel__k1__k2__length_scale':length_scale_init}
# dynamics_params_factory = {'estimator__kernel__length_scale':length_scale_init}

regressor = WrappedPlaidSklearnRegressor(reg, **config['regressor_mach']['plaid_params'], dynamics_params_factory = dynamics_params_factory)

target_regressor = PlaidTransformedTargetRegressor(
    regressor=regressor,
    transformer=postprocessor,
    transformed_target_feature_id=config['pca_mach']['plaid_params']['in_features_identifiers']
)
target_regressor

,regressor,WrappedPlaidS...om_state=42)))
,transformer,WrappedPlaidS...components=2))
,transformed_target_feature_id,"[{'base_name': 'Base_2_2', 'name': 'mach', 'type': 'field'}]"
,kernel,1**2 * Matern...noise_level=1)
,alpha,1e-10
,optimizer,'fmin_l_bfgs_b'
,n_restarts_optimizer,1
,normalize_y,False
,copy_X_train,True
,n_targets,None
,random_state,42


In [44]:
pipeline = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("regressor", target_regressor),
    ]
)
pipeline

,steps,"[('preprocessor', ...), ('regressor', ...)]"
,transform_input,None
,memory,None
,verbose,False
,plaid_transformers,"[('input_scalar_scaler', ...), ('pca_nodes', ...)]"
,remainder_feature_id,"[{'base_name': 'Base_2_2', 'name': 'mach', 'type': 'field'}]"
,feature_range,"(0, ...)"
,copy,True
,clip,False
,n_components,2
,copy,True


In [45]:
dataset_pred = preprocessor.fit_transform(dataset_train)
print(dataset_pred[0].get_scalar("angle_in"))
print(dataset_pred[0].get_scalar("mach_out"))
print(dataset_pred[0].get_scalar("reduced_nodes_*"))

0.9224137931034485
0.26369863013698547
[ 10.31073591 -15.88660491]


In [46]:
print(pipeline.get_params()['preprocessor__pca_nodes__sklearn_block__n_components'])
print(pipeline.get_params()['regressor__transformer__sklearn_block__n_components'])

2
2


In [47]:
pipeline.fit(dataset_train)
dataset_pred = pipeline.predict(dataset_train)
dataset_pred[0].get_field("mach", base_name = "Base_2_2")

array([0.31972743, 0.31974963, 0.3197783 , ..., 0.77907113, 0.77906518,
       0.77905851])

In [48]:
pipeline

,steps,"[('preprocessor', ...), ('regressor', ...)]"
,transform_input,None
,memory,None
,verbose,False
,plaid_transformers,"[('input_scalar_scaler', ...), ('pca_nodes', ...)]"
,remainder_feature_id,"[{'base_name': 'Base_2_2', 'name': 'mach', 'type': 'field'}]"
,feature_range,"(0, ...)"
,copy,True
,clip,False
,n_components,2
,copy,True


In [49]:
def objective(trial):
    # Suggest hyperparameters
    nodes_n_components = trial.suggest_int("preprocessor__pca_nodes__sklearn_block__n_components", 3, 4)
    mach_n_components = trial.suggest_int("regressor__transformer__sklearn_block__n_components", 4, 5)

    # Clone and configure pipeline
    pipeline_run = clone(pipeline)
    pipeline_run.set_params(
        preprocessor__pca_nodes__sklearn_block__n_components=nodes_n_components,
        regressor__transformer__sklearn_block__n_components=mach_n_components
    )

    cv = KFold(n_splits=3, shuffle=True, random_state=42)

    scores = []

    indices = np.arange(len(dataset_train))

    for train_idx, val_idx in cv.split(indices):

        dataset_cv_train_ = dataset_train[train_idx]
        dataset_cv_val_   = dataset_train[val_idx]

        pipeline_run.fit(dataset_cv_train_)

        # predicted_dataset_cv_val_ = pipeline_run.predict(dataset_cv_val_)

        score = pipeline_run.score(dataset_cv_val_)

        scores.append(score)

    return np.mean(scores)

In [50]:
print("dataset[0].get_field() =", dataset_train[0].get_field("mach", base_name="Base_2_2"))

dataset[0].get_field() = [0.39387196 0.39389698 0.39392865 ... 0.81002502 0.81000822 0.80999194]


In [51]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=4)

[I 2025-07-24 14:05:00,842] A new study created in memory with name: no-name-80ee348e-2527-4cf1-9f43-83c81ff734ce
[I 2025-07-24 14:05:04,124] Trial 0 finished with value: 0.9264574291180043 and parameters: {'preprocessor__pca_nodes__sklearn_block__n_components': 3, 'regressor__transformer__sklearn_block__n_components': 4}. Best is trial 0 with value: 0.9264574291180043.
[I 2025-07-24 14:05:07,086] Trial 1 finished with value: 0.9301528372077601 and parameters: {'preprocessor__pca_nodes__sklearn_block__n_components': 4, 'regressor__transformer__sklearn_block__n_components': 4}. Best is trial 1 with value: 0.9301528372077601.
[I 2025-07-24 14:05:09,648] Trial 2 finished with value: 0.9301528274123575 and parameters: {'preprocessor__pca_nodes__sklearn_block__n_components': 4, 'regressor__transformer__sklearn_block__n_components': 4}. Best is trial 1 with value: 0.9301528372077601.
[I 2025-07-24 14:05:11,939] Trial 3 finished with value: 0.926457398262003 and parameters: {'preprocessor__pc

In [52]:
print(study.best_params)
best_model = clone(pipeline).set_params(**study.best_params)
joblib.dump(best_model, 'artifacts/optuna_save.pkl')


{'preprocessor__pca_nodes__sklearn_block__n_components': 4, 'regressor__transformer__sklearn_block__n_components': 4}


['artifacts/optuna_save.pkl']

In [53]:
index = 5
optimized_pipeline = joblib.load('artifacts/optuna_save.pkl')
optimized_pipeline.fit(dataset_train)
dataset_pred = optimized_pipeline.predict(dataset_train)
print("mach_pred =", dataset_pred[index].get_field("mach", base_name = "Base_2_2"))
print("mach_ref =", dataset_train[index].get_field("mach", base_name = "Base_2_2"))
print("--")
print("angle_in =", dataset_pred[index].get_scalar("angle_in"))
print("mach_out =", dataset_pred[index].get_scalar("mach_out"))
print("reduced_nodes_* =", dataset_pred[index].get_scalar("reduced_nodes_*"))

mach_pred = [0.33169211 0.33173979 0.33180214 ... 0.72896549 0.72910477 0.72924111]
mach_ref = [0.3411916  0.34125071 0.34132835 ... 0.78406136 0.78400726 0.78394869]
--
angle_in = 0.6386494252873546
mach_out = 0.15684931506849198
reduced_nodes_* = [-31.13256395  26.35027297  -0.97654601  -0.93225342]


In [54]:
optimized_pipeline

,steps,"[('preprocessor', ...), ('regressor', ...)]"
,transform_input,None
,memory,None
,verbose,False
,plaid_transformers,"[('input_scalar_scaler', ...), ('pca_nodes', ...)]"
,remainder_feature_id,"[{'base_name': 'Base_2_2', 'name': 'mach', 'type': 'field'}]"
,feature_range,"(0, ...)"
,copy,True
,clip,False
,n_components,4
,copy,True


In [55]:
# print(optimized_pipeline.named_steps["preprocessor"].transformers_[0][1].sklearn_block_.get_params())
# print(optimized_pipeline.named_steps["preprocessor"].get_params())

In [56]:
print("GP length_scale =", optimized_pipeline.named_steps["regressor"].regressor_.sklearn_block_.estimators_[0].kernel_.get_params()['k1__k2__length_scale'])

GP length_scale = [4.57950198e+00 2.42415860e+00 2.50716442e+02 5.97813291e+01
 1.86570867e+05 3.93160802e+01]


In [57]:
print(optimized_pipeline[1].regressor)

WrappedPlaidSklearnRegressor(dynamics_params_factory={'estimator__kernel__k1__k2__length_scale': <function length_scale_init at 0x7fb1f1ad7ce0>},
                             in_features_identifiers=[{'name': 'angle_in',
                                                       'type': 'scalar'},
                                                      {'name': 'mach_out',
                                                       'type': 'scalar'},
                                                      {'name': 'reduced_nodes_*',
                                                       'type': 'scalar'}],
                             out_features_identifiers=[{'name': 'reduced_mach_*',
                                                        'type': 'scalar'}],
                             sklearn_block=MultiOutputRegressor(estimator=GaussianProcessRegressor(kernel=1**2 * Matern(length_scale=1, nu=2.5) + WhiteKernel(noise_level=1),
                                                                   

In [58]:
score = optimized_pipeline.score(dataset_train)
print("score =", score, "error =", 1. - score)


score = 0.9610575639025116 error = 0.038942436097488375


In [59]:
params = {'pca_nodes__sklearn_block__n_components': 4}
index = 5
preprocessor_ = clone(preprocessor)
preprocessor_.set_params(**params)
dataset_pred = preprocessor_.fit_transform(dataset_train)
print("angle_in =", dataset_pred[index].get_scalar("angle_in"))
print("mach_out =", dataset_pred[index].get_scalar("mach_out"))
print("reduced_nodes_* =", dataset_pred[index].get_scalar("reduced_nodes_*"))

angle_in = 0.6386494252873546
mach_out = 0.15684931506849198
reduced_nodes_* = [-31.13256395  26.35027297  -0.97654601  -0.93225342]


# GridSearchCV

In [60]:
param_grid = {
    'preprocessor__pca_nodes__sklearn_block__n_components': [2, 3],
    'regressor__transformer__sklearn_block__n_components': [4, 5],
}

search = GridSearchCV(pipeline, param_grid=param_grid, cv=3, verbose=3, error_score='raise')
search.fit(dataset_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV 1/3] END preprocessor__pca_nodes__sklearn_block__n_components=2, regressor__transformer__sklearn_block__n_components=4;, score=0.909 total time=   1.3s
[CV 2/3] END preprocessor__pca_nodes__sklearn_block__n_components=2, regressor__transformer__sklearn_block__n_components=4;, score=0.931 total time=   1.0s
[CV 3/3] END preprocessor__pca_nodes__sklearn_block__n_components=2, regressor__transformer__sklearn_block__n_components=4;, score=0.906 total time=   0.8s
[CV 1/3] END preprocessor__pca_nodes__sklearn_block__n_components=2, regressor__transformer__sklearn_block__n_components=5;, score=0.909 total time=   1.2s
[CV 2/3] END preprocessor__pca_nodes__sklearn_block__n_components=2, regressor__transformer__sklearn_block__n_components=5;, score=0.931 total time=   0.9s
[CV 3/3] END preprocessor__pca_nodes__sklearn_block__n_components=2, regressor__transformer__sklearn_block__n_components=5;, score=0.907 total time=   1.2s
[CV 

,estimator,Pipeline(step...onents=2))))])
,param_grid,"{'preprocessor__pca_node...arn_block__n_components': [2, 3], 'regressor__transformer...arn_block__n_components': [4, 5]}"
,scoring,None
,n_jobs,None
,refit,True
,cv,3
,verbose,3
,pre_dispatch,'2*n_jobs'
,error_score,'raise'
,return_train_score,False
,plaid_transformers,"[('input_scalar_scaler', ...), ('pca_nodes', ...)]"


In [61]:
best_params = search.best_params_
print("best_params =", best_params)
# best_model = search.best_estimator_
best_model = clone(pipeline).set_params(**best_params)
print(best_model.get_params()['preprocessor__pca_nodes__sklearn_block__n_components'])
print(best_model.get_params()['regressor__transformer__sklearn_block__n_components'])

best_params = {'preprocessor__pca_nodes__sklearn_block__n_components': 3, 'regressor__transformer__sklearn_block__n_components': 4}
3
4


In [62]:
best_model.fit(dataset_train)
score = best_model.score(dataset_train)
print("score =", score, "error =", 1. - score)

score = 0.9615036550944722 error = 0.03849634490552778


# COMMENTED TESTS

In [63]:
# test = WrappedPlaidSklearnTransformer(MinMaxScaler(), params = config['input_scalar_scaler'])
# test.fit(dataset_train)

In [64]:
# dd = test.transform(dataset_train)

# print(dataset_train)
# print(dd)

# dataset_2 = test.inverse_transform(dd)

# print(dataset_train[0].get_scalar("angle_in"))
# print(dd[0].get_scalar("angle_in"))
# print(dataset_2[0].get_scalar("angle_in"))
# print('===')
# print(dataset_train[0].get_scalar("mach_out"))
# print(dd[0].get_scalar("mach_out"))
# print(dataset_2[0].get_scalar("mach_out"))

In [65]:
# pca = WrappedPlaidSklearnTransformer(PCA(n_components = config['pca_nodes']['n_components']), params = config['pca_nodes'])
# pca.fit(dataset_train)
# dd = pca.transform(dataset_train)
# dataset_2 = pca.inverse_transform(dd)

# print(dataset_train[0].get_nodes(base_name = "Base_2_2"))
# print(dd[0].get_scalar("reduced_nodes_*"))
# print(dataset_2[0].get_nodes(base_name = "Base_2_2"))

In [66]:
# pca2 = WrappedPlaidSklearnTransformer(PCA(n_components = config['pca_mach']['n_components']), params = config['pca_mach'])
# pca2.fit(dataset_train)
# dd = pca2.transform(dataset_train)
# dataset_2 = pca2.inverse_transform(dd)

# print(dataset_train[0].get_field("mach", base_name = "Base_2_2"))
# print(dd[0].get_scalar("reduced_mach_*"))
# print(dataset_2[0].get_field("mach", base_name = "Base_2_2"))